In [ ]:
df_train= spark.read.parquet("s3://mukeshproj/Topic_modelling_pipeline_backup/topic_modelling_parquet_pipeline/shrink_data_piprline_backup/")

VBox()

Starting Spark application


In [ ]:
df_train.show(5)

In [ ]:
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '8g'),
                                        ('spark.driver.memory','15g'),
                                        ("spark.executor.cores", '2'),
                                        ("spark.dynamicAllocation.enabled","False")])

In [1]:
df_train= spark.read.parquet("hdfs:///review_shrunk_pipeline_output/")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1649258015822_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
vectorized_tokens=df_train.select("features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# cache and checkpoint 
# localcheckpoint 

In [3]:
vectorized_tokens.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|            features|
+--------------------+
|(10000,[0,1,5,6,1...|
|(10000,[1,7,11,13...|
|(10000,[0,2,6,7,1...|
|(10000,[0,1,3,4,5...|
|(10000,[0,11,16,4...|
+--------------------+
only showing top 5 rows

In [5]:
vectorized_tokens.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27

In [6]:
vectorized_tokens.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3548711

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 10
lda = LDA(k=num_topics, maxIter=1)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

In [ ]:
# extract vocabulary from CountVectorizer
vocab = countVectors.vocabulary
topics = model.describeTopics()   
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

In [ ]:
def plot_top_words(model, feature_names, n_top_words, title):
    #Modified from SKlearn
    fig, axes = plt.subplots(2, 5, figsize=(15, 7))
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 15})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=15)
        ax.tick_params(bottom=False)
        ax.set(xticklabels=[])

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
plot_top_words(model,features, 20,'Topics in LDa')